In [ ]:
import concurrent.futures
import logging
import os
import sqlite3
import requests
import time
from pathlib import Path

import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.exc import OperationalError, IntegrityError
from sp2ts import dt2sp

from lib.constants import SAVE_DIR, df_bm_units
from lib.data.utils import (
    add_bm_unit_type,
    parse_boal_from_physical_data,
    parse_fpn_from_physical_data, logger, N_POOL_INSTANCES, add_utc_timezone,
)

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

MAX_RETRIES = 1


def run_boa(
    start_date,
    end_date,
    units,
    chunk_size_in_days=7,
    database_engine=None,
    cache=True,
    multiprocess=True,
    pull_data_once=False,
):
    """
    Collects data from the ElexonAPI, saved as a local feather file, does some preprocessing and then places in
    an SQLite DB.

    Only collects data for specified units, to keep things fast. Uses multiprocessing to grab all units in parallel.
    """

    if database_engine is None:
        database_engine = create_engine("sqlite:///phys_data.db", echo=False)

    interval = pd.Timedelta(days=chunk_size_in_days)

    chunk_start = start_date
    chunk_end = start_date + interval
    logger.info(f"{chunk_start=} to {chunk_end=} ({end_date=})")
    while chunk_end <= end_date:
        logger.info(f"{chunk_start} to {chunk_end}")
        t1 = time.time()
        fetch_and_load_one_chunk(
            start_date=str(chunk_start),
            end_date=str(chunk_end),
            unit_ids=units,
            database_engine=database_engine,
            cache=cache,
            multiprocess=multiprocess,
            pull_data_once=pull_data_once,
        )
        t2 = time.time()
        logger.info(f"{(t2 - t1) / 60} minutes for {interval}")
        chunk_start = chunk_end
        chunk_end += interval


def write_fpn_to_db(df_fpn, database_engine) -> bool:
    """Write the FPN df to DB"""

    logger.info(f"Writing {len(df_fpn)} to FPN database")

    try:
        with database_engine.connect() as connection:
            df_fpn.to_sql("fpn", connection, if_exists="append", index_label="unit")
        return True
    except OperationalError:
        return False


def write_boal_to_db(df_boal, database_engine) -> bool:
    """Write the BOAL df to DB, falling back to a row-by-row load if the load of the whole df fails.

    This can happen because at boundaries between SPs, the same BOAL can be reported in multiple SP's. For instance,
    if the BOAL is 00:40 -> 01.05, it will be reported in two SPs, and so we can end up trying to load the same
    BOAL twice.
    """

    logger.info(f"Writing {len(df_boal)} to BOA database")

    try:
        with database_engine.connect() as connection:
            # Potential issue here with duplicate BOALs nuking the whole write. This can happen because
            # BOALs are extended across SPs
            try:
                df_boal.to_sql("boal", connection, if_exists="append", index_label="unit")
            except (sqlite3.IntegrityError, IntegrityError) as e:
                logging.warning(e)
                # Try and write them one at at time
                for i in range(len(df_boal)):
                    try:
                        df_boal.iloc[i].to_sql(
                            "boal",
                            con=connection,
                            if_exists="append",
                            index_label="unit",
                        )
                    except IntegrityError as e:
                        logging.warning(e)
                        pass
        return True
    except OperationalError:
        return False


def fetch_and_load_one_chunk(
    start_date,
    end_date,
    unit_ids,
    database_engine,
    cache=True,
    multiprocess=True,
    pull_data_once=False,
):
    """Fetch and load FPN and BOAL data for `start_date` to `end_date` for units in `unit_ids"""
    # TODO clean up the preprocessing of data here
    logger = logging.getLogger(__name__)
    logger.setLevel(logging.DEBUG)

    logger.info(f"{start_date}-{end_date}")

    df = fetch_physical_data(
        start_date=start_date,
        end_date=end_date,
        save_dir=SAVE_DIR,
        cache=cache,
        unit_ids=unit_ids,
        multiprocess=multiprocess,
        pull_data_once=pull_data_once,
    )

    df = df.rename(columns={"bmUnitID": "Unit"})
    df["timeFrom"], df["timeTo"] = df["timeFrom"].apply(pd.to_datetime), df["timeTo"].apply(
        pd.to_datetime
    )

    df = add_bm_unit_type(df, df_bm_units=df_bm_units)

    df_fpn, df_boal = parse_fpn_from_physical_data(df), parse_boal_from_physical_data(df)

    logger.debug(f"there are {len(df_fpn)} FPNS")
    logger.debug(f"there are {len(df_boal)} BOAs")

    logger.debug(f"Selecting wind units only")
    if len(df_boal) > 0:
        df_boal = df_boal[df_boal["Fuel Type"] == "WIND"]
    if len(df_fpn) > 0:
        df_fpn = df_fpn[df_fpn["Fuel Type"] == "WIND"]

    logger.debug(f'Selecting wind units only')

    if len(df_boal) > 0:
        df_boal = df_boal[df_boal["Fuel Type"] == "WIND"]
    if len(df_fpn) > 0:
        df_fpn = df_fpn[df_fpn["Fuel Type"] == "WIND"]

    # Duplicates can occur from multiple SP's reporting the same BOAL
    df_boal = df_boal.drop_duplicates(
        subset=["timeFrom", "timeTo", "Accept ID", "levelFrom", "levelTo"]
    )

    # DB Locking collisions between processes necessitate a retry loop
    fpn_success = write_fpn_to_db(df_fpn, database_engine)
    retries = 0
    while not fpn_success and retries < MAX_RETRIES:
        logger.info("Retrying FPN after sleep")
        time.sleep(np.random.randint(1, 20))
        fpn_success = write_fpn_to_db(df_fpn, database_engine)
        retries += 1

    # Separated these because pandas autocommits, so FPN could end up being retried unecessarily
    # if subsequent BOAL write has failed!
    boal_success = write_boal_to_db(df_boal, database_engine)
    retries = 0
    while not boal_success and retries < MAX_RETRIES:
        logger.info("Retrying BOAL after sleep")
        time.sleep(np.random.randint(1, 20))
        boal_success = write_boal_to_db(df_boal, database_engine)
        retries += 1


def call_physbm_api(start_date, end_date, unit=None):
    """Thin wrapper to allow kwarg passing with starmap"""
    logger.info(f"Calling BOAS API for {unit}")

    # Nedd to call PNs and BOALs separately in new API

    # "https://data.elexon.co.uk/bmrs/api/v1/balancing/physical/all?dataset={dataset}&settlementDate={settlementDate}&settlementPeriod={settlementPeriod}&format=json"
    datetimes = pd.date_range(start_date, end_date, freq="30min")
    data_df = []
    for datetime in datetimes:
        logger.info(f"Getting PN from {datetime}")

        datetime = add_utc_timezone(datetime)

        date, sp = dt2sp(datetime)
        url = f"https://data.elexon.co.uk/bmrs/api/v1/balancing/physical/all?dataset=PN&settlementDate={date}&settlementPeriod={sp}"
        if unit is not None:
            url = url + f"&bmUnit={unit}"
        url = url + "&format=json"

        r = requests.get(url)

        data_one_settlement_period_df = pd.DataFrame(r.json()["data"])
        data_df.append(data_one_settlement_period_df)

    data_pn_df = pd.concat(data_df)

    datetimes = pd.date_range(start_date, end_date, freq="30min")
    data_df = []
    for datetime in datetimes:
        logger.info(f"Getting BOALF from {datetime}")
        boalf_end_datetime = (datetime + pd.Timedelta(minutes=30)).tz_localize(None)
        boalf_start_datetime = (datetime - pd.Timedelta(minutes=30)).tz_localize(None)
        url = f"https://data.elexon.co.uk/bmrs/api/v1/datasets/BOALF?from={boalf_start_datetime}&to={boalf_end_datetime}"
        if unit is not None:
            url = url + f"&bmUnit={unit}"
        url = url + "&format=json"

        r = requests.get(url)

        data_one_settlement_period_df = pd.DataFrame(r.json()["data"])
        data_df.append(data_one_settlement_period_df)

    data_boa_df = pd.concat(data_df)

    # rename bmUnit to bmUnitID
    data_pn_df.rename(columns={"bmUnit": "bmUnitID"}, inplace=True)
    data_boa_df.rename(columns={"bmUnit": "bmUnitID"}, inplace=True)

    # drop dataset column
    data_boa_df.drop(columns=["nationalGridBmUnit"], inplace=True)
    data_pn_df.drop(columns=["nationalGridBmUnit"], inplace=True)
    data_boa_df.drop(columns=["settlementPeriodTo"], inplace=True)
    data_boa_df.drop(columns=["amendmentFlag"], inplace=True)
    data_boa_df.drop(columns=["storFlag"], inplace=True)

    # rename LevelFrom to bidOfferLevelFrom
    data_pn_df.rename(columns={"dataset": "recordType"}, inplace=True)
    data_boa_df.rename(columns={"dataset": "recordType"}, inplace=True)
    data_boa_df.rename(columns={"acceptanceNumber": "Accept ID"}, inplace=True)
    data_boa_df.rename(columns={"settlementPeriodFrom": "settlementPeriod"}, inplace=True)
    data_boa_df.rename(columns={"deemedBoFlag": "deemedBidOfferFlag"}, inplace=True)
    data_boa_df.rename(columns={"rrFlag": "rrScheduleFlag"}, inplace=True)

    data_df = pd.concat([data_boa_df, data_pn_df], axis=0)
    data_df['local_datetime'] = pd.to_datetime(data_df['timeFrom'])

    # remove anything after end_date
    data_df = data_df[data_df['local_datetime'] <= end_date]

    return data_df


def fetch_physical_data(
    start_date, end_date, save_dir: Path, cache=True, unit_ids=None, multiprocess=False, pull_data_once: bool = False
):
    """From a brief visual inspection, this returns data that looks the same as the stuff I downloaded manually"""
    if cache:
        file_name = save_dir / f"{start_date}-{end_date}.fthr"
        if file_name.exists():
            return pd.read_feather(file_name)

    if (unit_ids is not None) and (not pull_data_once):
        if multiprocess:
            unit_dfs = []
            with concurrent.futures.ThreadPoolExecutor(
                max_workers=int(os.getenv("N_POOL_INSTANCES", N_POOL_INSTANCES))
            ) as executor:

                tasks = [executor.submit(call_physbm_api, start_date, end_date, unit) for unit in unit_ids]

                for future in concurrent.futures.as_completed(tasks):
                    data = future.result()
                    unit_dfs.append(data)

        else:
            unit_dfs = []
            for i, unit in enumerate(unit_ids):
                logger.info(f"Calling API PHYBMDATA for {unit} ({i}/{len(unit_ids)}) " f"{start_date=} {end_date=}")
                unit_dfs.append(call_physbm_api(start_date, end_date, unit))

        df = pd.concat(unit_dfs)
    else:
        df = call_physbm_api(start_date=start_date, end_date=end_date)
        if unit_ids is not None:
            df = df[df["bmUnitID"].isin(unit_ids)]

    if cache:
        df.reset_index(drop=True).to_feather(file_name)

    return df

In [4]:
import concurrent.futures
import logging
import os
import sqlite3
import requests
import time
from pathlib import Path

import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.exc import OperationalError, IntegrityError
from sp2ts import dt2sp

start_date = '01/01/2025'
end_date = '02/01/2025'

def add_utc_timezone(datetime):
    """ Add utc timezone to datetime. """
    if datetime.tzinfo is None:
        datetime = datetime.tz_localize('UTC')
    else:
        datetime = datetime.tz_convert('UTC')
    return datetime

# Nedd to call PNs and BOALs separately in new API

# "https://data.elexon.co.uk/bmrs/api/v1/balancing/physical/all?dataset={dataset}&settlementDate={settlementDate}&settlementPeriod={settlementPeriod}&format=json"
datetimes = pd.date_range(start_date, end_date, freq="30min")
data_df = []
for datetime in datetimes:
    # logger.info(f"Getting PN from {datetime}")

    datetime = add_utc_timezone(datetime)

    date, sp = dt2sp(datetime)

In [40]:
pd.set_option("display.max_columns", None)
def get_data_from_elexon_api(url):

    # url = f"https://data.elexon.co.uk/bmrs/api/v1/balancing/physical/all?dataset=PN&settlementDate={date}&settlementPeriod={sp}"
    r = requests.get(url)
    if r.status_code == 404:
        print(f'Data not found for {url}')
        return
    df = pd.DataFrame(r.json())

    return df

url = 'https://data.elexon.co.uk/bmrs/api/v1/reference/bmunits/all'
r = get_data_from_elexon_api(url)

In [69]:
r.nationalGridBmUnit.duplicated().sum()

np.int64(1)

In [73]:
r

,nationalGridBmUnit,elexonBmUnit,eic,fuelType,leadPartyName,bmUnitType,fpnFlag,bmUnitName,leadPartyId,demandCapacity,generationCapacity,productionOrConsumptionFlag,transmissionLossFactor,workingDayCreditAssessmentImportCapability,nonWorkingDayCreditAssessmentImportCapability,workingDayCreditAssessmentExportCapability,nonWorkingDayCreditAssessmentExportCapability,creditQualifyingStatus,demandInProductionFlag,gspGroupId,gspGroupName,interconnectorId
0,ABERU-1,E_ABERDARE,None,None,UK Power Reserve Limited,E,True,Aberdare Power Station,UKPR,0.000,15.400,C,0.0162007,0.000,0.000,6.160,6.160,True,False,_K,South Wales,None
1,ABRBO-1,T_ABRBO-1,48W00000ABRBO-19,WIND,Aberdeen Offshore Wind Farm,T,True,ABRBO-1,ABERDEEN,-2.000,99.000,P,-0.0301340,-0.800,-0.800,39.600,39.600,True,False,None,None,None
2,ABRTW-1,E_ABRTW-1,48W00000ABRTW-1Z,WIND,Npower Commercial Gas Limited,E,True,Auchrobert Wind Farm,NPOWER02,-10.000,36.000,C,-0.0088069,-4.000,-4.000,14.400,14.400,True,False,_N,South Scotland,None
3,ACHLW-1,C__PSMAR001,None,WIND,Smartestenergy Limited,S,True,C__PSMAR001-AAA-ACH-183,SMARTEST,0.000,9.978,C,-0.0301340,0.000,0.000,3.186,3.186,False,False,_P,North Scotland,None
4,ACHRW-1,T_ACHRW-1,48W00000ACHRW-1O,WIND,Mobius Renewables Gen. (GB2,T,True,AChruach Wind Farm,AIRGEN2,-2.000,42.600,P,-0.0301340,-0.800,-0.800,17.040,17.040,True,False,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2694,YEP-SYM01,2__MSYMB000,None,None,E.ON Next Energy Ltd,G,True,2__MSYMB000,NITTWO01,0.000,0.000,C,-0.0090952,0.000,0.000,0.000,0.000,False,False,_M,Yorkshire Electricity,None
2695,YEP-TFE01,2__MTGPL000,None,None,TotalEnergies Gas & Power Ltd,G,True,2__MTGPL000,TFEGP,-201.515,64.000,C,-0.0051169,-117.221,-82.984,37.229,26.355,False,False,_M,Yorkshire,None
2696,YEP-TIL01,2__MTILL000,None,None,British Gas Trading Ltd,G,True,2__MTILL000,BRITGAS,-403.373,0.000,C,-0.0051169,-192.732,-202.453,0.000,0.000,False,False,_M,Yorkshire,None
2697,YEP-YEP01,2__MYELG000,None,None,npower limited,G,True,_M Default BM Yorkshire,YE,-26.020,0.000,C,-0.0072983,-14.103,-14.589,0.000,0.000,False,False,_M,Yorkshire Electricity,None


In [75]:
r.head()

,nationalGridBmUnit,elexonBmUnit,eic,fuelType,leadPartyName,bmUnitType,fpnFlag,bmUnitName,leadPartyId,demandCapacity,generationCapacity,productionOrConsumptionFlag,transmissionLossFactor,workingDayCreditAssessmentImportCapability,nonWorkingDayCreditAssessmentImportCapability,workingDayCreditAssessmentExportCapability,nonWorkingDayCreditAssessmentExportCapability,creditQualifyingStatus,demandInProductionFlag,gspGroupId,gspGroupName,interconnectorId
0,ABERU-1,E_ABERDARE,None,None,UK Power Reserve Limited,E,True,Aberdare Power Station,UKPR,0.000,15.400,C,0.0162007,0.000,0.000,6.160,6.160,True,False,_K,South Wales,None
1,ABRBO-1,T_ABRBO-1,48W00000ABRBO-19,WIND,Aberdeen Offshore Wind Farm,T,True,ABRBO-1,ABERDEEN,-2.000,99.000,P,-0.0301340,-0.800,-0.800,39.600,39.600,True,False,None,None,None
2,ABRTW-1,E_ABRTW-1,48W00000ABRTW-1Z,WIND,Npower Commercial Gas Limited,E,True,Auchrobert Wind Farm,NPOWER02,-10.000,36.000,C,-0.0088069,-4.000,-4.000,14.400,14.400,True,False,_N,South Scotland,None
3,ACHLW-1,C__PSMAR001,None,WIND,Smartestenergy Limited,S,True,C__PSMAR001-AAA-ACH-183,SMARTEST,0.000,9.978,C,-0.0301340,0.000,0.000,3.186,3.186,False,False,_P,North Scotland,None
4,ACHRW-1,T_ACHRW-1,48W00000ACHRW-1O,WIND,Mobius Renewables Gen. (GB2,T,True,AChruach Wind Farm,AIRGEN2,-2.000,42.600,P,-0.0301340,-0.800,-0.800,17.040,17.040,True,False,None,None,None


In [98]:
data_df

,recordType,settlementDate,settlementPeriod,timeFrom,timeTo,levelFrom,levelTo,Accept ID,acceptanceTime,deemedBidOfferFlag,soFlag,rrScheduleFlag,bmUnitID,local_datetime
749,BOALF,2024-12-31,48,2024-12-31T23:33:00Z,2024-12-31T23:36:00Z,-6,-4,1055.0,2024-12-31T23:30:00Z,False,False,False,2__AMRCY001,2024-12-31 23:33:00+00:00
692,BOALF,2024-12-31,48,2024-12-31T23:36:00Z,2024-12-31T23:45:00Z,-4,-4,1055.0,2024-12-31T23:30:00Z,False,False,False,2__AMRCY001,2024-12-31 23:36:00+00:00
602,BOALF,2024-12-31,48,2024-12-31T23:45:00Z,2024-12-31T23:48:00Z,-4,-6,1055.0,2024-12-31T23:30:00Z,False,False,False,2__AMRCY001,2024-12-31 23:45:00+00:00
603,BOALF,2024-12-31,48,2024-12-31T23:45:00Z,2024-12-31T23:47:00Z,-4,-4,1056.0,2024-12-31T23:41:00Z,False,False,False,2__AMRCY001,2024-12-31 23:45:00+00:00
558,BOALF,2024-12-31,48,2024-12-31T23:47:00Z,2024-12-31T23:50:00Z,-4,-6,1056.0,2024-12-31T23:41:00Z,False,False,False,2__AMRCY001,2024-12-31 23:47:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,PN,2024-12-31,48,2024-12-31T23:30:00Z,2025-01-01T00:00:00Z,0,0,NaN,NaN,NaN,NaN,NaN,None,2024-12-31 23:30:00+00:00
127,PN,2024-12-31,48,2024-12-31T23:30:00Z,2025-01-01T00:00:00Z,0,0,NaN,NaN,NaN,NaN,NaN,None,2024-12-31 23:30:00+00:00
128,PN,2024-12-31,48,2024-12-31T23:30:00Z,2025-01-01T00:00:00Z,0,0,NaN,NaN,NaN,NaN,NaN,None,2024-12-31 23:30:00+00:00
129,PN,2024-12-31,48,2024-12-31T23:30:00Z,2025-01-01T00:00:00Z,0,0,NaN,NaN,NaN,NaN,NaN,None,2024-12-31 23:30:00+00:00


In [100]:
data_df_x = data_df.merge(r, how = 'inner', left_on = 'bmUnitID', right_on = 'elexonBmUnit')
data_df_x[(data_df_x['bmUnitName'].str.len() > 20) & (data_df_x['fuelType'] == 'WIND')].sort_values('bmUnitName')[['bmUnitID','leadPartyName','bmUnitName','levelFrom']].iloc[40:60]

,bmUnitID,leadPartyName,bmUnitName,levelFrom
2171,T_DBAWO-3,Doggerbank OWF Project 1,Dogger Bank A Offshore WF 3,0
2172,T_DBAWO-4,Doggerbank OWF Project 1,Dogger Bank A Offshore WF 4,0
2173,T_DBAWO-5,Doggerbank OWF Project 1,Dogger Bank A Offshore WF 5,0
2193,T_DOREW-1,Dorenell Windfarm Limited,Dorenell Wind Farm One,96
2194,T_DOREW-2,Dorenell Windfarm Limited,Dorenell Wind Farm Two,81
2174,T_DDGNO-1,Dudgeon Offshore Wind Ltd,Dudgeon Generator BMU 1,107
2175,T_DDGNO-2,Dudgeon Offshore Wind Ltd,Dudgeon Generator BMU 2,83
2176,T_DDGNO-3,Dudgeon Offshore Wind Ltd,Dudgeon Generator BMU 3,94
2177,T_DDGNO-4,Dudgeon Offshore Wind Ltd,Dudgeon Generator BMU 4,88
2207,T_EAAO-1,East Anglia One Limited,East Anglia One Offshore 1,340


In [96]:
data_df_x

,recordType,settlementDate,settlementPeriod,timeFrom,timeTo,levelFrom,levelTo,Accept ID,acceptanceTime,deemedBidOfferFlag,soFlag,rrScheduleFlag,bmUnitID,local_datetime,nationalGridBmUnit,elexonBmUnit,eic,fuelType,leadPartyName,bmUnitType,fpnFlag,bmUnitName,leadPartyId,demandCapacity,generationCapacity,productionOrConsumptionFlag,transmissionLossFactor,workingDayCreditAssessmentImportCapability,nonWorkingDayCreditAssessmentImportCapability,workingDayCreditAssessmentExportCapability,nonWorkingDayCreditAssessmentExportCapability,creditQualifyingStatus,demandInProductionFlag,gspGroupId,gspGroupName,interconnectorId
0,BOALF,2024-12-31,48,2024-12-31T23:33:00Z,2024-12-31T23:36:00Z,-6,-4,1055.0,2024-12-31T23:30:00Z,False,False,False,2__AMRCY001,2024-12-31 23:33:00+00:00,AG-AOCT01,2__AMRCY001,None,None,Octopus Energy Limited,S,True,2__AMRCY001,MERCURY,-12.200,0.000,C,0.0042196,-3.525,-3.797,0.000,0.000,False,False,_A,Eastern,None
1,BOALF,2024-12-31,48,2024-12-31T23:36:00Z,2024-12-31T23:45:00Z,-4,-4,1055.0,2024-12-31T23:30:00Z,False,False,False,2__AMRCY001,2024-12-31 23:36:00+00:00,AG-AOCT01,2__AMRCY001,None,None,Octopus Energy Limited,S,True,2__AMRCY001,MERCURY,-12.200,0.000,C,0.0042196,-3.525,-3.797,0.000,0.000,False,False,_A,Eastern,None
2,BOALF,2024-12-31,48,2024-12-31T23:45:00Z,2024-12-31T23:48:00Z,-4,-6,1055.0,2024-12-31T23:30:00Z,False,False,False,2__AMRCY001,2024-12-31 23:45:00+00:00,AG-AOCT01,2__AMRCY001,None,None,Octopus Energy Limited,S,True,2__AMRCY001,MERCURY,-12.200,0.000,C,0.0042196,-3.525,-3.797,0.000,0.000,False,False,_A,Eastern,None
3,BOALF,2024-12-31,48,2024-12-31T23:45:00Z,2024-12-31T23:47:00Z,-4,-4,1056.0,2024-12-31T23:41:00Z,False,False,False,2__AMRCY001,2024-12-31 23:45:00+00:00,AG-AOCT01,2__AMRCY001,None,None,Octopus Energy Limited,S,True,2__AMRCY001,MERCURY,-12.200,0.000,C,0.0042196,-3.525,-3.797,0.000,0.000,False,False,_A,Eastern,None
4,BOALF,2024-12-31,48,2024-12-31T23:47:00Z,2024-12-31T23:50:00Z,-4,-6,1056.0,2024-12-31T23:41:00Z,False,False,False,2__AMRCY001,2024-12-31 23:47:00+00:00,AG-AOCT01,2__AMRCY001,None,None,Octopus Energy Limited,S,True,2__AMRCY001,MERCURY,-12.200,0.000,C,0.0042196,-3.525,-3.797,0.000,0.000,False,False,_A,Eastern,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4436,PN,2024-12-31,48,2024-12-31T23:30:00Z,2025-01-01T00:00:00Z,0,0,NaN,NaN,NaN,NaN,NaN,None,2024-12-31 23:30:00+00:00,TONG-3,None,None,NPSHYD,None,None,None,None,None,None,None,None,None,None,None,None,None,False,False,None,None,None
4437,PN,2024-12-31,48,2024-12-31T23:30:00Z,2025-01-01T00:00:00Z,0,0,NaN,NaN,NaN,NaN,NaN,None,2024-12-31 23:30:00+00:00,TUMB-1,None,48W000000TUMB-1G,NPSHYD,None,None,None,None,None,None,None,None,None,None,None,None,None,False,False,None,None,None
4438,PN,2024-12-31,48,2024-12-31T23:30:00Z,2025-01-01T00:00:00Z,0,0,NaN,NaN,NaN,NaN,NaN,None,2024-12-31 23:30:00+00:00,TUMB-2,None,48W000000TUMB-2E,NPSHYD,None,None,None,None,None,None,None,None,None,None,None,None,None,False,False,None,None,None
4439,PN,2024-12-31,48,2024-12-31T23:30:00Z,2025-01-01T00:00:00Z,0,0,NaN,NaN,NaN,NaN,NaN,None,2024-12-31 23:30:00+00:00,WINN-1,None,None,CCGT,None,None,None,None,None,None,None,None,None,None,None,None,None,False,False,None,None,None


In [ ]:
r[r['bmUnitName'].astype(str).str.lower().str.contains('scarweather')]

,nationalGridBmUnit,elexonBmUnit,eic,fuelType,leadPartyName,bmUnitType,fpnFlag,bmUnitName,leadPartyId,demandCapacity,generationCapacity,productionOrConsumptionFlag,transmissionLossFactor,workingDayCreditAssessmentImportCapability,nonWorkingDayCreditAssessmentImportCapability,workingDayCreditAssessmentExportCapability,nonWorkingDayCreditAssessmentExportCapability,creditQualifyingStatus,demandInProductionFlag,gspGroupId,gspGroupName,interconnectorId


In [ ]:
'Cromer' in r['bmUnitName'].astype()

False

In [56]:
r.head()[['leadPartyName','bmUnitName']]

,leadPartyName,bmUnitName
0,UK Power Reserve Limited,Aberdare Power Station
1,Aberdeen Offshore Wind Farm,ABRBO-1
2,Npower Commercial Gas Limited,Auchrobert Wind Farm
3,Smartestenergy Limited,C__PSMAR001-AAA-ACH-183
4,Mobius Renewables Gen. (GB2,AChruach Wind Farm


In [34]:
data = r.json()
pd.DataFrame(data)

,nationalGridBmUnit,elexonBmUnit,eic,fuelType,leadPartyName,bmUnitType,fpnFlag,bmUnitName,leadPartyId,demandCapacity,...,transmissionLossFactor,workingDayCreditAssessmentImportCapability,nonWorkingDayCreditAssessmentImportCapability,workingDayCreditAssessmentExportCapability,nonWorkingDayCreditAssessmentExportCapability,creditQualifyingStatus,demandInProductionFlag,gspGroupId,gspGroupName,interconnectorId
0,ABERU-1,E_ABERDARE,None,None,UK Power Reserve Limited,E,True,Aberdare Power Station,UKPR,0.000,...,0.0162007,0.000,0.000,6.160,6.160,True,False,_K,South Wales,None
1,ABRBO-1,T_ABRBO-1,48W00000ABRBO-19,WIND,Aberdeen Offshore Wind Farm,T,True,ABRBO-1,ABERDEEN,-2.000,...,-0.0301340,-0.800,-0.800,39.600,39.600,True,False,None,None,None
2,ABRTW-1,E_ABRTW-1,48W00000ABRTW-1Z,WIND,Npower Commercial Gas Limited,E,True,Auchrobert Wind Farm,NPOWER02,-10.000,...,-0.0088069,-4.000,-4.000,14.400,14.400,True,False,_N,South Scotland,None
3,ACHLW-1,C__PSMAR001,None,WIND,Smartestenergy Limited,S,True,C__PSMAR001-AAA-ACH-183,SMARTEST,0.000,...,-0.0301340,0.000,0.000,3.186,3.186,False,False,_P,North Scotland,None
4,ACHRW-1,T_ACHRW-1,48W00000ACHRW-1O,WIND,Mobius Renewables Gen. (GB2,T,True,AChruach Wind Farm,AIRGEN2,-2.000,...,-0.0301340,-0.800,-0.800,17.040,17.040,True,False,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2694,YEP-SYM01,2__MSYMB000,None,None,E.ON Next Energy Ltd,G,True,2__MSYMB000,NITTWO01,0.000,...,-0.0090952,0.000,0.000,0.000,0.000,False,False,_M,Yorkshire Electricity,None
2695,YEP-TFE01,2__MTGPL000,None,None,TotalEnergies Gas & Power Ltd,G,True,2__MTGPL000,TFEGP,-201.515,...,-0.0051169,-117.221,-82.984,37.229,26.355,False,False,_M,Yorkshire,None
2696,YEP-TIL01,2__MTILL000,None,None,British Gas Trading Ltd,G,True,2__MTILL000,BRITGAS,-403.373,...,-0.0051169,-192.732,-202.453,0.000,0.000,False,False,_M,Yorkshire,None
2697,YEP-YEP01,2__MYELG000,None,None,npower limited,G,True,_M Default BM Yorkshire,YE,-26.020,...,-0.0072983,-14.103,-14.589,0.000,0.000,False,False,_M,Yorkshire Electricity,None


In [20]:
df_elexon_api.json().to_dataframe()

AttributeError: 'list' object has no attribute 'to_dataframe'

In [27]:
import json
json.load(df_elexon_api)

AttributeError: 'Response' object has no attribute 'read'

In [ ]:
# df_elexon_api.read_json()
pd.read_json(df_elexon_api)

ValueError: Invalid file path or buffer object type: <class 'list'>

In [1]:
# basic load BOA and FP data from elexon
import concurrent.futures
import logging
import os
import sqlite3
import requests
import time
from pathlib import Path

import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.exc import OperationalError, IntegrityError
from sp2ts import dt2sp

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)



def add_utc_timezone(datetime):
    """ Add utc timezone to datetime. """
    if datetime.tzinfo is None:
        datetime = datetime.tz_localize('UTC')
    else:
        datetime = datetime.tz_convert('UTC')
    return datetime


def call_physbm_api(start_date, end_date, unit=None):
    """Thin wrapper to allow kwarg passing with starmap"""
    logger.info(f"Calling BOAS API for {unit}")

    # Nedd to call PNs and BOALs separately in new API

    # "https://data.elexon.co.uk/bmrs/api/v1/balancing/physical/all?dataset={dataset}&settlementDate={settlementDate}&settlementPeriod={settlementPeriod}&format=json"
    datetimes = pd.date_range(start_date, end_date, freq="30min")
    data_df = []
    for datetime in datetimes:
        logger.info(f"Getting PN from {datetime}")

        datetime = add_utc_timezone(datetime)

        date, sp = dt2sp(datetime)
        url = f"https://data.elexon.co.uk/bmrs/api/v1/balancing/physical/all?dataset=PN&settlementDate={date}&settlementPeriod={sp}"
        if unit is not None:
            url = url + f"&bmUnit={unit}"
        url = url + "&format=json"

        r = requests.get(url)

        data_one_settlement_period_df = pd.DataFrame(r.json()["data"])
        data_df.append(data_one_settlement_period_df)

    data_pn_df = pd.concat(data_df)

    datetimes = pd.date_range(start_date, end_date, freq="30min")
    data_df = []
    for datetime in datetimes:
        logger.info(f"Getting BOALF from {datetime}")
        boalf_end_datetime = (datetime + pd.Timedelta(minutes=30)).tz_localize(None)
        boalf_start_datetime = (datetime - pd.Timedelta(minutes=30)).tz_localize(None)
        url = f"https://data.elexon.co.uk/bmrs/api/v1/datasets/BOALF?from={boalf_start_datetime}&to={boalf_end_datetime}"
        if unit is not None:
            url = url + f"&bmUnit={unit}"
        url = url + "&format=json"

        r = requests.get(url)

        data_one_settlement_period_df = pd.DataFrame(r.json()["data"])
        data_df.append(data_one_settlement_period_df)

    data_boa_df = pd.concat(data_df)

    # rename bmUnit to bmUnitID
    data_pn_df.rename(columns={"bmUnit": "bmUnitID"}, inplace=True)
    data_boa_df.rename(columns={"bmUnit": "bmUnitID"}, inplace=True)

    # drop dataset column
    data_boa_df.drop(columns=["nationalGridBmUnit"], inplace=True)
    data_pn_df.drop(columns=["nationalGridBmUnit"], inplace=True)
    data_boa_df.drop(columns=["settlementPeriodTo"], inplace=True)
    data_boa_df.drop(columns=["amendmentFlag"], inplace=True)
    data_boa_df.drop(columns=["storFlag"], inplace=True)

    # rename LevelFrom to bidOfferLevelFrom
    data_pn_df.rename(columns={"dataset": "recordType"}, inplace=True)
    data_boa_df.rename(columns={"dataset": "recordType"}, inplace=True)
    data_boa_df.rename(columns={"acceptanceNumber": "Accept ID"}, inplace=True)
    data_boa_df.rename(columns={"settlementPeriodFrom": "settlementPeriod"}, inplace=True)
    data_boa_df.rename(columns={"deemedBoFlag": "deemedBidOfferFlag"}, inplace=True)
    data_boa_df.rename(columns={"rrFlag": "rrScheduleFlag"}, inplace=True)

    data_df = pd.concat([data_boa_df, data_pn_df], axis=0)
    data_df['local_datetime'] = pd.to_datetime(data_df['timeFrom'])

    # remove anything after end_date
    data_df = data_df[data_df['local_datetime'] <= end_date]

    return data_df

start_date = '01/01/2025'
end_date = '01/01/2025'

# specific wind units only - bug fix with retries and also add aprallel api calls
# df_bm_units = pd.read_csv('BMU.csv')
# wind_units = df_bm_units[df_bm_units["FUEL TYPE"] == "WIND"]["SETT_BMU_ID"].unique()
# data_df=pd.DataFrame()
# for unit in wind_units:
#     data_df = pd.concat([data_df, call_physbm_api(start_date, end_date, unit)])

data_df = call_physbm_api(start_date, end_date)
data_df=data_df.sort_values(['recordType','bmUnitID','timeFrom'])

INFO:__main__:Calling BOAS API for None
INFO:__main__:Getting PN from 2025-01-01 00:00:00
INFO:__main__:Getting BOALF from 2025-01-01 00:00:00


In [62]:
data_df

,recordType,settlementDate,settlementPeriod,timeFrom,timeTo,levelFrom,levelTo,Accept ID,acceptanceTime,deemedBidOfferFlag,soFlag,rrScheduleFlag,bmUnitID,local_datetime
749,BOALF,2024-12-31,48,2024-12-31T23:33:00Z,2024-12-31T23:36:00Z,-6,-4,1055.0,2024-12-31T23:30:00Z,False,False,False,2__AMRCY001,2024-12-31 23:33:00+00:00
692,BOALF,2024-12-31,48,2024-12-31T23:36:00Z,2024-12-31T23:45:00Z,-4,-4,1055.0,2024-12-31T23:30:00Z,False,False,False,2__AMRCY001,2024-12-31 23:36:00+00:00
602,BOALF,2024-12-31,48,2024-12-31T23:45:00Z,2024-12-31T23:48:00Z,-4,-6,1055.0,2024-12-31T23:30:00Z,False,False,False,2__AMRCY001,2024-12-31 23:45:00+00:00
603,BOALF,2024-12-31,48,2024-12-31T23:45:00Z,2024-12-31T23:47:00Z,-4,-4,1056.0,2024-12-31T23:41:00Z,False,False,False,2__AMRCY001,2024-12-31 23:45:00+00:00
558,BOALF,2024-12-31,48,2024-12-31T23:47:00Z,2024-12-31T23:50:00Z,-4,-6,1056.0,2024-12-31T23:41:00Z,False,False,False,2__AMRCY001,2024-12-31 23:47:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,PN,2024-12-31,48,2024-12-31T23:30:00Z,2025-01-01T00:00:00Z,0,0,NaN,NaN,NaN,NaN,NaN,None,2024-12-31 23:30:00+00:00
127,PN,2024-12-31,48,2024-12-31T23:30:00Z,2025-01-01T00:00:00Z,0,0,NaN,NaN,NaN,NaN,NaN,None,2024-12-31 23:30:00+00:00
128,PN,2024-12-31,48,2024-12-31T23:30:00Z,2025-01-01T00:00:00Z,0,0,NaN,NaN,NaN,NaN,NaN,None,2024-12-31 23:30:00+00:00
129,PN,2024-12-31,48,2024-12-31T23:30:00Z,2025-01-01T00:00:00Z,0,0,NaN,NaN,NaN,NaN,NaN,None,2024-12-31 23:30:00+00:00


In [36]:
data_df[data_df['bmUnitID'] == '2__AMRCY001']

,recordType,settlementDate,settlementPeriod,timeFrom,timeTo,levelFrom,levelTo,Accept ID,acceptanceTime,deemedBidOfferFlag,soFlag,rrScheduleFlag,bmUnitID,local_datetime
749,BOALF,2024-12-31,48,2024-12-31T23:33:00Z,2024-12-31T23:36:00Z,-6,-4,1055.0,2024-12-31T23:30:00Z,False,False,False,2__AMRCY001,2024-12-31 23:33:00+00:00
692,BOALF,2024-12-31,48,2024-12-31T23:36:00Z,2024-12-31T23:45:00Z,-4,-4,1055.0,2024-12-31T23:30:00Z,False,False,False,2__AMRCY001,2024-12-31 23:36:00+00:00
602,BOALF,2024-12-31,48,2024-12-31T23:45:00Z,2024-12-31T23:48:00Z,-4,-6,1055.0,2024-12-31T23:30:00Z,False,False,False,2__AMRCY001,2024-12-31 23:45:00+00:00
603,BOALF,2024-12-31,48,2024-12-31T23:45:00Z,2024-12-31T23:47:00Z,-4,-4,1056.0,2024-12-31T23:41:00Z,False,False,False,2__AMRCY001,2024-12-31 23:45:00+00:00
558,BOALF,2024-12-31,48,2024-12-31T23:47:00Z,2024-12-31T23:50:00Z,-4,-6,1056.0,2024-12-31T23:41:00Z,False,False,False,2__AMRCY001,2024-12-31 23:47:00+00:00
150,PN,2024-12-31,48,2024-12-31T23:30:00Z,2025-01-01T00:00:00Z,-6,-6,NaN,NaN,NaN,NaN,NaN,2__AMRCY001,2024-12-31 23:30:00+00:00


In [27]:
data_df

,recordType,settlementDate,settlementPeriod,timeFrom,timeTo,levelFrom,levelTo,Accept ID,acceptanceTime,deemedBidOfferFlag,soFlag,rrScheduleFlag,bmUnitID,local_datetime
749,BOALF,2024-12-31,48,2024-12-31T23:33:00Z,2024-12-31T23:36:00Z,-6,-4,1055.0,2024-12-31T23:30:00Z,False,False,False,2__AMRCY001,2024-12-31 23:33:00+00:00
692,BOALF,2024-12-31,48,2024-12-31T23:36:00Z,2024-12-31T23:45:00Z,-4,-4,1055.0,2024-12-31T23:30:00Z,False,False,False,2__AMRCY001,2024-12-31 23:36:00+00:00
602,BOALF,2024-12-31,48,2024-12-31T23:45:00Z,2024-12-31T23:48:00Z,-4,-6,1055.0,2024-12-31T23:30:00Z,False,False,False,2__AMRCY001,2024-12-31 23:45:00+00:00
603,BOALF,2024-12-31,48,2024-12-31T23:45:00Z,2024-12-31T23:47:00Z,-4,-4,1056.0,2024-12-31T23:41:00Z,False,False,False,2__AMRCY001,2024-12-31 23:45:00+00:00
558,BOALF,2024-12-31,48,2024-12-31T23:47:00Z,2024-12-31T23:50:00Z,-4,-6,1056.0,2024-12-31T23:41:00Z,False,False,False,2__AMRCY001,2024-12-31 23:47:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,PN,2024-12-31,48,2024-12-31T23:30:00Z,2025-01-01T00:00:00Z,0,0,NaN,NaN,NaN,NaN,NaN,None,2024-12-31 23:30:00+00:00
127,PN,2024-12-31,48,2024-12-31T23:30:00Z,2025-01-01T00:00:00Z,0,0,NaN,NaN,NaN,NaN,NaN,None,2024-12-31 23:30:00+00:00
128,PN,2024-12-31,48,2024-12-31T23:30:00Z,2025-01-01T00:00:00Z,0,0,NaN,NaN,NaN,NaN,NaN,None,2024-12-31 23:30:00+00:00
129,PN,2024-12-31,48,2024-12-31T23:30:00Z,2025-01-01T00:00:00Z,0,0,NaN,NaN,NaN,NaN,NaN,None,2024-12-31 23:30:00+00:00
